In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil
import os

src = '/content/drive/MyDrive/lot_3_clean'
dst = '/content/folder'

if os.path.exists(dst):
    shutil.rmtree(dst)

shutil.copytree(src, dst)

'/content/folder'

In [ ]:
pip install opencv-python numpy Pillow tqdm

In [ ]:
#@title Partitionnement Chunking avec paramètres (contraste minimum, taille minimum, start y_very_top pour les chunks 000)

import numpy as np
from PIL import Image # PIL might still be needed by remove_light_background_rgba
from PIL import Image
import os
import cv2
import sys
import time
from collections import Counter
from tqdm.notebook import tqdm # Use tqdm for progress bars if in Colab/Jupyter
import traceback # Import traceback for better error details

# --- Configuration (Keep as is) ---
INPUT_FOLDER = "/content/folder"
OUTPUT_FOLDER = "/content/folder_partitioned_merged"
CHUNK_TARGET_LINES = 13
OVERLAP_LINES = 2
PIXEL_MARGIN = 5
INITIAL_LINE_SPACING = 87
SPACING_SEARCH_RANGE = 3
DEFAULT_LIGHTNESS_THRESHOLD = 190
DEFAULT_SATURATION_LIMIT = 60
ALPHA_THRESHOLD = 10
MERGE_MIN_HEIGHT = 15
MERGE_MAX_HEIGHT = 100
MERGE_CONTRAST_THRESHOLD = 11.0

# --- Helper Functions (Keep as is) ---
def calculate_image_std_dev(img_array_gray):
    if img_array_gray is None or img_array_gray.size == 0: return 0.0
    try: return np.std(img_array_gray)
    except Exception as e: print(f"Warning: Error calculating std dev: {e}", file=sys.stderr); return 0.0

def remove_light_background_rgba(img_path, lightness_thresh, saturation_limit):
    try:
        pil_img = Image.open(img_path).convert('RGB')
        img_rgb = np.array(pil_img)
        img_hsv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HSV)
        h, s, v = cv2.split(img_hsv)
        _, lightness_mask = cv2.threshold(v, lightness_thresh, 255, cv2.THRESH_BINARY)
        _, saturation_mask = cv2.threshold(s, saturation_limit, 255, cv2.THRESH_BINARY_INV)
        background_mask = cv2.bitwise_or(lightness_mask, saturation_mask)
        alpha_channel = cv2.bitwise_not(background_mask)
        img_rgba = np.dstack((img_rgb, alpha_channel))
        return img_rgba
    except FileNotFoundError: print(f"  Error: Input file not found at '{img_path}'", file=sys.stderr); return None
    except Exception as e: print(f"  Error removing background for {img_path}: {e}", file=sys.stderr); return None

def calculate_overlap(alpha_channel, offset, spacing, img_height, img_width):
    total_overlap = 0
    if spacing <= 0: return float('inf')
    for y in range(offset, img_height, spacing):
        if y < img_height:
            line_pixels = alpha_channel[y, :]
            overlap_on_line = np.sum(line_pixels > ALPHA_THRESHOLD)
            total_overlap += overlap_on_line
    return total_overlap

def find_best_line_grid(alpha_channel, initial_spacing, spacing_range, img_height, img_width):
    best_offset, best_spacing, min_score = -1, -1, float('inf')
    print(f"  Searching spacing from {initial_spacing - spacing_range} to {initial_spacing + spacing_range}")
    for spacing in range(initial_spacing - spacing_range, initial_spacing + spacing_range + 1):
        if spacing <= 0: continue
        for offset in range(spacing):
            score = calculate_overlap(alpha_channel, offset, spacing, img_height, img_width)
            if score < min_score:
                min_score, best_offset, best_spacing = score, offset, spacing
    return best_offset, best_spacing, min_score

# --- *** MODIFIED Main Processing Function *** ---
def process_partition_merge_image(image_path, input_base_dir, output_base_dir, config):
    """
    Detects lines, partitions into overlapping chunks (ensuring first chunk starts at y=0),
    merges small final non-blank chunks, saves chunks, returns chunk line counts.
    """
    print(f"\nProcessing: {image_path}")
    chunk_line_counts = []

    try:
        # --- 1. Detect Lines ---
        img_rgba = remove_light_background_rgba(image_path,
                                                config['lightness_thresh'],
                                                config['saturation_limit'])
        if img_rgba is None: return None # Skip if background removal failed

        alpha_channel = img_rgba[:, :, 3]
        img_height, img_width = alpha_channel.shape[:2]
        print(f"  Image dimensions: {img_width}x{img_height}")

        best_offset, best_spacing, min_score = find_best_line_grid(
            alpha_channel,
            config['initial_spacing'],
            config['spacing_range'],
            img_height,
            img_width
        )

        if best_offset == -1: print(f"  Warning: Could not find grid for {image_path}. Skipping."); return None

        detected_lines_y = sorted([y for y in range(best_offset, img_height, best_spacing)])
        num_lines = len(detected_lines_y)
        print(f"  Detected {num_lines} lines with spacing {best_spacing} starting at offset {best_offset}.")

        if num_lines == 0: print(f"  Warning: Zero lines detected. Skipping."); return None
        # (Handle other num_lines edge cases as before)
        if num_lines < config['overlap_lines'] and num_lines > 0: print(f"  Warning: Fewer lines ({num_lines}) than overlap ({config['overlap_lines']}).")

        # --- 2. Load Original Image ---
        img_original = cv2.imread(image_path, cv2.IMREAD_COLOR)
        if img_original is None: print(f"  Error: Could not read original image {image_path}. Skipping."); return None
        original_height, original_width = img_original.shape[:2]

        # --- 3. Define Chunk Boundaries and Crop with Merge Logic ---
        chunk_index = 0 # IMPORTANT: Use this to track the *output* chunk index
        step_size = config['chunk_lines'] - config['overlap_lines']
        if step_size <= 0: print(f"  Error: Chunk lines <= overlap lines. Skipping."); return None

        relative_path = os.path.relpath(os.path.dirname(image_path), input_base_dir)
        output_subdir = os.path.join(output_base_dir, relative_path)
        os.makedirs(output_subdir, exist_ok=True)
        base_filename = os.path.splitext(os.path.basename(image_path))[0]

        prev_chunk_data = None
        prev_chunk_path = None
        prev_chunk_line_count = 0
        saved_chunk_count = 0
        start_indices = list(range(0, num_lines, step_size))

        for i, start_line_idx in enumerate(start_indices):
            end_line_idx = start_line_idx + config['chunk_lines'] - 1
            actual_end_line_idx = min(end_line_idx, num_lines - 1)

            if start_line_idx > actual_end_line_idx: continue

            y_start_line = detected_lines_y[start_line_idx]
            y_end_line = detected_lines_y[actual_end_line_idx]

            # Determine the top pixel boundary for the crop
            if i == 0: # Check if it's the very first chunk being generated (index 0 in the loop)
                y_top_pixel = 0 # Force the first chunk to start at the top edge (y=0)
                print(f"  Chunk {chunk_index}: Setting y_top_pixel=0 (First Chunk)")
            else:
                # For subsequent chunks, use the detected line minus the margin
                y_top_pixel = max(0, y_start_line - config['pixel_margin'])

            # Bottom boundary calculation remains the same
            y_bottom_pixel = min(original_height, y_end_line + config['pixel_margin'])

            # Validate slice dimensions
            if y_bottom_pixel <= y_top_pixel:
                print(f"  Warning: Invalid slice dimensions (y_top={y_top_pixel}, y_bottom={y_bottom_pixel}) for chunk {chunk_index}. Skipping.")
                # Note: We don't increment chunk_index here if skipping, as it refers to the output index
                continue # Skip this iteration

            # --- Crop the current chunk ---
            current_chunk_data = img_original[y_top_pixel:y_bottom_pixel, :]
            if current_chunk_data is None or current_chunk_data.size == 0:
                 print(f"  Warning: Cropping resulted in empty image for chunk {chunk_index}. Skipping.")
                 # Don't increment chunk_index if skipping
                 continue

            # Prepare info for the current chunk
            current_chunk_height = current_chunk_data.shape[0]
            current_chunk_line_count = actual_end_line_idx - start_line_idx + 1
            current_output_filename = f"{base_filename}_chunk_{chunk_index:03d}.png" # Use chunk_index for filename
            current_output_path = os.path.join(output_subdir, current_output_filename)

            is_last_chunk = (i == len(start_indices) - 1)

            # --- Logic for Saving/Merging (Remains largely the same, uses chunk_index for filenames) ---
            if is_last_chunk:
                print(f"  Processing last potential chunk (Output Index {chunk_index}, Height {current_chunk_height}px)")
                if config['merge_min_height'] <= current_chunk_height <= config['merge_max_height']:
                    print(f"    Last chunk is small. Checking contrast...")
                    gray_chunk = cv2.cvtColor(current_chunk_data, cv2.COLOR_BGR2GRAY)
                    std_dev = calculate_image_std_dev(gray_chunk)
                    print(f"    Contrast (Std Dev): {std_dev:.2f}")
                    if std_dev > config['merge_contrast_thresh']:
                        print(f"    High contrast. Merging with previous chunk.")
                        if prev_chunk_data is not None:
                            # Merge: Overwrite the previous file
                            combined_chunk = np.vstack((prev_chunk_data, current_chunk_data))
                            try:
                                cv2.imwrite(prev_chunk_path, combined_chunk)
                                print(f"    Successfully saved merged chunk to: {prev_chunk_path}")
                                chunk_line_counts.append(prev_chunk_line_count + current_chunk_line_count) # Update combined count
                                saved_chunk_count += 1 # Count the saved merged file
                                prev_chunk_data = None # Clear merged data
                            except Exception as e: print(f"    Error saving merged chunk {prev_chunk_path}: {e}")
                        else: # First chunk was small and contrasty
                            print("    No previous chunk to merge with. Saving independently.")
                            try:
                                cv2.imwrite(current_output_path, current_chunk_data)
                                print(f"      Saved chunk {chunk_index}: {current_output_path}")
                                chunk_line_counts.append(current_chunk_line_count)
                                saved_chunk_count += 1
                            except Exception as e: print(f"      Error saving chunk {chunk_index}: {e}")
                    else: # Low contrast small last chunk
                        print(f"    Low contrast. Saving separately.")
                        if prev_chunk_data is not None: # Save previous if exists
                            try:
                                cv2.imwrite(prev_chunk_path, prev_chunk_data)
                                print(f"      Saved previous chunk {chunk_index-1}: {prev_chunk_path}")
                                chunk_line_counts.append(prev_chunk_line_count)
                                saved_chunk_count += 1
                            except Exception as e: print(f"      Error saving previous chunk {chunk_index-1}: {e}")
                        try: # Save current small blank chunk
                            cv2.imwrite(current_output_path, current_chunk_data)
                            print(f"      Saved current chunk {chunk_index}: {current_output_path}")
                            chunk_line_counts.append(current_chunk_line_count)
                            saved_chunk_count += 1
                        except Exception as e: print(f"      Error saving current chunk {chunk_index}: {e}")
                        prev_chunk_data = None
                else: # Last chunk not small
                    print(f"    Last chunk height ({current_chunk_height}px) outside merge range. Saving separately.")
                    if prev_chunk_data is not None: # Save previous if exists
                        try:
                            cv2.imwrite(prev_chunk_path, prev_chunk_data)
                            print(f"      Saved previous chunk {chunk_index-1}: {prev_chunk_path}")
                            chunk_line_counts.append(prev_chunk_line_count)
                            saved_chunk_count += 1
                        except Exception as e: print(f"      Error saving previous chunk {chunk_index-1}: {e}")
                    try: # Save current (last) chunk
                        cv2.imwrite(current_output_path, current_chunk_data)
                        print(f"      Saved current chunk {chunk_index}: {current_output_path}")
                        chunk_line_counts.append(current_chunk_line_count)
                        saved_chunk_count += 1
                    except Exception as e: print(f"      Error saving current chunk {chunk_index}: {e}")
                    prev_chunk_data = None

            else: # Not the last chunk
                if prev_chunk_data is not None: # Save the previous chunk now
                    try:
                        cv2.imwrite(prev_chunk_path, prev_chunk_data)
                        # print(f"    Saved chunk {chunk_index-1}: {prev_chunk_path}")
                        chunk_line_counts.append(prev_chunk_line_count)
                        saved_chunk_count += 1
                    except Exception as e: print(f"    Error saving chunk {chunk_index-1} to {prev_chunk_path}: {e}")

                # Store current chunk data for the *next* iteration or final save
                prev_chunk_data = current_chunk_data
                prev_chunk_path = current_output_path
                prev_chunk_line_count = current_chunk_line_count

            # Increment the output chunk index *after* successfully processing/cropping a chunk
            chunk_index += 1

        # --- Final Check (Safeguard - should be covered by is_last_chunk logic) ---
        if prev_chunk_data is not None:
             print("  Performing final save check for potentially unsaved penultimate chunk...")
             try:
                 cv2.imwrite(prev_chunk_path, prev_chunk_data)
                 print(f"    Saved final pending chunk {chunk_index-1}: {prev_chunk_path}")
                 chunk_line_counts.append(prev_chunk_line_count)
                 saved_chunk_count += 1
             except Exception as e: print(f"    Error saving final pending chunk {chunk_index-1}: {e}")

        if saved_chunk_count == 0: print(f"  Warning: No chunks were successfully saved for {image_path}."); return None
        print(f"  Successfully generated {saved_chunk_count} final chunk file(s).")
        return chunk_line_counts

    except Exception as e:
        print(f"  *** An unexpected error occurred processing {image_path}: {e} ***", file=sys.stderr)
        traceback.print_exc()
        return None

# --- Main Execution (Keep as is) ---
if __name__ == "__main__":
    if not os.path.isdir(INPUT_FOLDER): print(f"Error: Input folder not found at '{INPUT_FOLDER}'", file=sys.stderr); sys.exit(1)
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    print(f"Input folder: {INPUT_FOLDER}")
    print(f"Output folder for partitions: {OUTPUT_FOLDER}")
    # ... (rest of the main execution block remains the same) ...
    print(f"Partition settings: Target Lines={CHUNK_TARGET_LINES}, Overlap={OVERLAP_LINES}, Margin={PIXEL_MARGIN}px")
    print(f"Line Spacing Search: {INITIAL_LINE_SPACING - SPACING_SEARCH_RANGE}px to {INITIAL_LINE_SPACING + SPACING_SEARCH_RANGE}px")
    print(f"Final Chunk Merge: Height {MERGE_MIN_HEIGHT}-{MERGE_MAX_HEIGHT}px, Merge if StdDev > {MERGE_CONTRAST_THRESHOLD:.2f}")

    config = {
        'chunk_lines': CHUNK_TARGET_LINES, 'overlap_lines': OVERLAP_LINES, 'pixel_margin': PIXEL_MARGIN,
        'initial_spacing': INITIAL_LINE_SPACING, 'spacing_range': SPACING_SEARCH_RANGE,
        'lightness_thresh': DEFAULT_LIGHTNESS_THRESHOLD, 'saturation_limit': DEFAULT_SATURATION_LIMIT,
        'merge_min_height': MERGE_MIN_HEIGHT, 'merge_max_height': MERGE_MAX_HEIGHT,
        'merge_contrast_thresh': MERGE_CONTRAST_THRESHOLD,
    }

    all_files = []
    for root, dirs, files in os.walk(INPUT_FOLDER):
        dirs.sort(); files.sort()
        for filename in files:
            if filename.lower().endswith(".png"): all_files.append(os.path.join(root, filename))

    print(f"\nFound {len(all_files)} PNG files to process.")
    overall_chunk_line_distribution = Counter()
    processed_files_count = 0; failed_files_count = 0; total_chunks_generated = 0

    for file_path in tqdm(all_files, desc="Partitioning Images"):
        result_counts = process_partition_merge_image(file_path, INPUT_FOLDER, OUTPUT_FOLDER, config)
        if result_counts is not None:
            processed_files_count += 1
            overall_chunk_line_distribution.update(result_counts)
            total_chunks_generated += len(result_counts)
        else: failed_files_count += 1

    print("\n--- Partitioning Complete ---")
    print(f"Successfully processed {processed_files_count} image files.")
    print(f"Failed or skipped {failed_files_count} image files.")
    print(f"Total final chunk files generated: {total_chunks_generated}")
    if total_chunks_generated > 0:
        print("\nDistribution of lines per final chunk file:")
        sorted_distribution = sorted(overall_chunk_line_distribution.items())
        for num_lines, count in sorted_distribution: print(f"  Chunks with {num_lines} lines: {count}")
    else: print("\nNo chunk files were generated.")
    print(f"\nPartitioned images saved in: {OUTPUT_FOLDER}")

Input folder: /content/folder
Output folder for partitions: /content/folder_partitioned_merged
Partition settings: Target Lines=13, Overlap=2, Margin=5px
Line Spacing Search: 84px to 90px
Final Chunk Merge: Height 15-100px, Merge if StdDev > 11.00

Found 262 PNG files to process.


Partitioning Images:   0%|          | 0/262 [00:00<?, ?it/s]


Processing: /content/folder/Commentaire Locke, Essay, II, 21 (AGREG ext. 2019 - note _ 16)/page_01.png
  Image dimensions: 2481x2598
  Searching spacing from 84 to 90
  Detected 29 lines with spacing 88 starting at offset 46.
  Chunk 0: Setting y_top_pixel=0 (First Chunk)
  Processing last potential chunk (Output Index 2, Height 538px)
    Last chunk height (538px) outside merge range. Saving separately.
      Saved previous chunk 1: /content/folder_partitioned_merged/Commentaire Locke, Essay, II, 21 (AGREG ext. 2019 - note _ 16)/page_01_chunk_001.png
      Saved current chunk 2: /content/folder_partitioned_merged/Commentaire Locke, Essay, II, 21 (AGREG ext. 2019 - note _ 16)/page_01_chunk_002.png
  Successfully generated 3 final chunk file(s).

Processing: /content/folder/Commentaire Locke, Essay, II, 21 (AGREG ext. 2019 - note _ 16)/page_02.png
  Image dimensions: 2481x3000
  Searching spacing from 84 to 90
  Detected 36 lines with spacing 84 starting at offset 4.
  Chunk 0: Setting

In [ ]:
#@title Add red lines

import cv2
import numpy as np
from PIL import Image
import os
import sys
import time
from tqdm.notebook import tqdm # Use tqdm for progress bars if in Colab/Jupyter

# --- Configuration ---
FOLDER_PATH = "/content/folder"  # <--- CHANGE THIS if your folder is elsewhere
INITIAL_LINE_SPACING = 87      # Your observed consistent spacing
SPACING_SEARCH_RANGE = 2       # Test spacings: initial +/- this range (e.g., 1 -> [86, 87, 88])
# Background Removal Parameters (Tune these based on your previous results)
DEFAULT_LIGHTNESS_THRESHOLD = 190
DEFAULT_SATURATION_LIMIT = 60
# Line Drawing Parameters
LINE_COLOR = (0, 0, 255)  # Red in BGR format
LINE_THICKNESS = 1        # Thickness of the drawn line (can be thin now)
# Overlap calculation threshold (pixels with alpha > this are considered 'text')
ALPHA_THRESHOLD = 10

# --- Background Removal Function (Adapted from your code) ---
def remove_light_background_rgba(img_path, lightness_thresh, saturation_limit):
    """
    Loads an image, removes light/unsaturated background,
    and returns an RGBA NumPy array.
    """
    try:
        pil_img = Image.open(img_path).convert('RGB')
        img_rgb = np.array(pil_img)
        img_hsv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HSV)
        h, s, v = cv2.split(img_hsv)

        _, lightness_mask = cv2.threshold(v, lightness_thresh, 255, cv2.THRESH_BINARY)
        _, saturation_mask = cv2.threshold(s, saturation_limit, 255, cv2.THRESH_BINARY_INV)

        background_mask = cv2.bitwise_or(lightness_mask, saturation_mask)
        alpha_channel = cv2.bitwise_not(background_mask)

        img_rgba = np.dstack((img_rgb, alpha_channel))
        return img_rgba

    except Exception as e:
        print(f"  Error removing background for {img_path}: {e}", file=sys.stderr)
        return None

# --- Overlap Calculation Function ---
def calculate_overlap(alpha_channel, offset, spacing, img_height, img_width):
    """
    Calculates the overlap score for a given grid configuration.
    Overlap is defined as the sum of alpha values > ALPHA_THRESHOLD on candidate lines.
    """
    total_overlap = 0
    if spacing <= 0: # Prevent division by zero or infinite loop
        return float('inf')

    for y in range(offset, img_height, spacing):
        if y < img_height: # Ensure y is within bounds
            line_pixels = alpha_channel[y, :]
            # Count pixels considered 'text' on this line
            overlap_on_line = np.sum(line_pixels > ALPHA_THRESHOLD)
            total_overlap += overlap_on_line

            # Optional: Weight overlap by line length? Probably not needed if lines are full width.
            # total_overlap += overlap_on_line / img_width

    return total_overlap

# --- Find Best Grid Function ---
def find_best_line_grid(alpha_channel, initial_spacing, spacing_range, img_height, img_width):
    """
    Finds the best (offset, spacing) combination that minimizes text overlap.
    """
    best_offset = -1
    best_spacing = -1
    min_score = float('inf')

    # Iterate through possible spacings
    for spacing in range(initial_spacing - spacing_range, initial_spacing + spacing_range + 1):
        if spacing <= 0: continue # Skip invalid spacings

        # Iterate through possible offsets for this spacing
        # Using TQDM here for inner loop progress if desired
        # for offset in tqdm(range(spacing), desc=f"Testing Spacing {spacing}", leave=False):
        for offset in range(spacing):
            score = calculate_overlap(alpha_channel, offset, spacing, img_height, img_width)

            if score < min_score:
                min_score = score
                best_offset = offset
                best_spacing = spacing
            # Optional: Handle ties or near-ties if needed, e.g., prefer middle offset
            # elif score == min_score and abs(offset - spacing//2) < abs(best_offset - best_spacing//2):
            #     best_offset = offset
            #     best_spacing = spacing


    return best_offset, best_spacing, min_score

# --- Main Processing Function ---
def process_image_file(image_path):
    """
    Processes a single image: removes background, finds best line grid,
    draws lines on original, and overwrites it.
    """
    print(f"Processing: {image_path}")
    try:
        # 1. Remove Background
        img_rgba = remove_light_background_rgba(image_path,
                                                DEFAULT_LIGHTNESS_THRESHOLD,
                                                DEFAULT_SATURATION_LIMIT)
        if img_rgba is None:
            return False # Indicate failure

        alpha_channel = img_rgba[:, :, 3]
        img_height, img_width = alpha_channel.shape[:2]
        print(f"  Image dimensions: {img_width}x{img_height}")

        # 2. Find Best Line Grid
        start_time = time.time()
        best_offset, best_spacing, min_score = find_best_line_grid(
            alpha_channel,
            INITIAL_LINE_SPACING,
            SPACING_SEARCH_RANGE,
            img_height,
            img_width
        )
        end_time = time.time()
        print(f"  Found best grid: Offset={best_offset}, Spacing={best_spacing}, Score={min_score:.0f} (took {end_time - start_time:.2f}s)")

        if best_offset == -1:
            print(f"  Warning: Could not find a valid grid for {image_path}")
            return False

        # 3. Generate Final Line Coordinates
        line_y_coords = [y for y in range(best_offset, img_height, best_spacing)]
        print(f"  Generated {len(line_y_coords)} lines.")

        # 4. Load Original Image and Draw Lines
        img_original_color = cv2.imread(image_path, cv2.IMREAD_COLOR)
        if img_original_color is None:
             print(f"  Error: Could not re-read original image {image_path} for drawing.")
             return False # Should not happen if background removal worked, but safety check

        img_with_lines = img_original_color.copy()
        for y in line_y_coords:
            cv2.line(img_with_lines, (0, y), (img_width, y), LINE_COLOR, LINE_THICKNESS)

        # 5. Overwrite Original File
        success = cv2.imwrite(image_path, img_with_lines)
        if success:
            print(f"  Successfully saved modified image: {image_path}")
            return True
        else:
            print(f"  Error: Failed to save modified image: {image_path}")
            return False

    except Exception as e:
        print(f"  *** An error occurred processing {image_path}: {e} ***", file=sys.stderr)
        # Optionally add more detailed traceback
        # import traceback
        # traceback.print_exc()
        return False # Indicate failure

# --- Main Execution ---
if __name__ == "__main__":
    if not os.path.isdir(FOLDER_PATH):
        print(f"Error: Folder not found at '{FOLDER_PATH}'", file=sys.stderr)
        sys.exit(1)

    print(f"Starting line detection using grid optimization in folder: {FOLDER_PATH}")
    print(f"Initial spacing guess: {INITIAL_LINE_SPACING}, Search range: +/- {SPACING_SEARCH_RANGE}")
    print(f"Overlap threshold (alpha): > {ALPHA_THRESHOLD}")

    processed_count = 0
    failed_count = 0
    all_files = []
    for root, dirs, files in os.walk(FOLDER_PATH):
        for filename in files:
            if filename.lower().endswith(".png"):
                 all_files.append(os.path.join(root, filename))

    print(f"Found {len(all_files)} PNG files to process.")

    # Use tqdm for overall progress bar
    for file_path in tqdm(all_files, desc="Processing Images"):
        if process_image_file(file_path):
            processed_count += 1
        else:
            failed_count += 1
        # Optional: Add a small delay if hitting resource limits, though unlikely needed
        # time.sleep(0.05)

    print("\n--- Processing Complete ---")
    print(f"Successfully processed and overwrote: {processed_count} images.")
    print(f"Failed or skipped: {failed_count} images.")
    if failed_count > 0:
        print("Check console output or error logs for details on failures.")
    print("\nIMPORTANT: Check a few output images visually to ensure the line detection worked as expected.")
    print("The script has overwritten the original files with versions marked with red lines based on the optimized grid.")

Starting line detection using grid optimization in folder: /content/folder
Initial spacing guess: 87, Search range: +/- 1
Overlap threshold (alpha): > 10
Found 402 PNG files to process.


Processing Images:   0%|          | 0/402 [00:00<?, ?it/s]

Processing: /content/folder/Peut on vivre en paix avec son inconscient ? (AGREG interne 2021 - note : 17)/page_08.png
  Image dimensions: 2481x3508
  Found best grid: Offset=87, Spacing=88, Score=19 (took 0.11s)
  Generated 39 lines.
  Successfully saved modified image: /content/folder/Peut on vivre en paix avec son inconscient ? (AGREG interne 2021 - note : 17)/page_08.png
Processing: /content/folder/Peut on vivre en paix avec son inconscient ? (AGREG interne 2021 - note : 17)/page_06.png
  Image dimensions: 2481x2999
  Found best grid: Offset=11, Spacing=86, Score=0 (took 0.09s)
  Generated 35 lines.
  Successfully saved modified image: /content/folder/Peut on vivre en paix avec son inconscient ? (AGREG interne 2021 - note : 17)/page_06.png
Processing: /content/folder/Peut on vivre en paix avec son inconscient ? (AGREG interne 2021 - note : 17)/page_07.png
  Image dimensions: 2481x3508
  Found best grid: Offset=4, Spacing=87, Score=4 (took 0.11s)
  Generated 41 lines.
  Successfully 

KeyboardInterrupt: 

In [ ]:
pip install opencv-python numpy tqdm

In [ ]:
#@title contrastes suspects

import cv2
import numpy as np
from PIL import Image
import os
import sys
from google.colab import output as colab_output
from IPython.display import display, clear_output
import ipywidgets as widgets
from io import BytesIO
from tqdm.notebook import tqdm
import traceback # Import traceback for detailed error printing

# --- Configuration ---
CHUNK_FOLDER = "/content/drive/MyDrive/folder_partitioned_merged_clean"
LOW_CONTRAST_THRESHOLD = 20.0 # Adjust if needed
BORDER_SIZE = 5
BORDER_COLOR = [0, 0, 0]

# --- Global Variables for State ---
suspect_files = []
current_suspect_index = -1
# Flag to prevent re-finding files on cell re-run if already done
files_found = False

# --- Helper Function: Calculate Image Standard Deviation (Same as before) ---
def calculate_image_std_dev(image_path):
    try:
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            # print(f"Warning: Could not read image {image_path}", file=sys.stderr)
            return None
        if img.size == 0:
             # print(f"Warning: Image {image_path} is empty.", file=sys.stderr)
             return 0.0
        std_dev = np.std(img)
        return std_dev
    except Exception as e:
        print(f"Error calculating std dev for {image_path}: {e}", file=sys.stderr)
        return None

# --- Function to Find Suspect Files (Same as before) ---
def find_suspect_chunk_files(folder_path, threshold):
    global files_found
    print(f"Scanning {folder_path} for low-contrast chunks (std dev < {threshold})...")
    suspects = []
    all_png_files = []
    for root, _, files in os.walk(folder_path):
        for filename in files:
            if filename.lower().endswith(".png"):
                all_png_files.append(os.path.join(root, filename))

    if not all_png_files:
        print("No PNG files found in the specified folder.")
        files_found = True # Mark as found (even if empty) to prevent re-scan
        return []

    for file_path in tqdm(all_png_files, desc="Analyzing Chunks"):
        std_dev = calculate_image_std_dev(file_path)
        # Check explicitly for None in case of read error
        if std_dev is not None and std_dev < threshold:
            suspects.append(file_path)

    print(f"Found {len(suspects)} potential low-contrast suspect files.")
    files_found = True # Mark as found
    return suspects

# --- Function to Display a Suspect Image and Controls ---
def display_next_suspect(b=None): # Add optional button argument (though not used directly here)
    """Displays the next suspect image and the Delete/Keep buttons."""
    global current_suspect_index, suspect_files

    current_suspect_index += 1
    print(f"\nEntering display_next_suspect, new index: {current_suspect_index}") # DEBUG

    # Clear previous output in the designated area
    with out:
        clear_output(wait=True)
        print(f"Attempting to display suspect index: {current_suspect_index}") # DEBUG

        if not suspect_files or current_suspect_index >= len(suspect_files):
            print("✅ No more suspect files to review or list is empty.")
            delete_button.disabled = True
            keep_button.disabled = True
            return

        # Check if file exists *before* trying to load
        while current_suspect_index < len(suspect_files):
            current_file = suspect_files[current_suspect_index]
            if os.path.exists(current_file):
                 print(f"File exists: {current_file}") # DEBUG
                 break # Found an existing file
            else:
                 print(f"Info: File '{current_file}' not found (already deleted?). Skipping.")
                 current_suspect_index += 1 # Skip to next index
        else: # Loop finished without finding an existing file
            print("✅ No more existing suspect files found.")
            delete_button.disabled = True
            keep_button.disabled = True
            return


        # --- Try loading and displaying the current file ---
        current_file = suspect_files[current_suspect_index]
        print(f"Reviewing ({current_suspect_index + 1}/{len(suspect_files)}): {current_file}")

        try:
            # Load image with OpenCV
            print(f"  Attempting cv2.imread: {current_file}") # DEBUG
            img = cv2.imread(current_file)

            if img is None:
                # CRITICAL: Handle imread failure gracefully
                print(f"❌ Error: cv2.imread failed to load image {current_file}. It might be corrupted or empty.")
                print("  Please check the file manually. Skipping this file.")
                # Display buttons again to allow skipping
                display(ui_layout)
                delete_button.disabled = True # Disable delete if we can't load it
                keep_button.disabled = False # Allow skipping
                filename_widget.value = f"<b>File:</b> {os.path.basename(current_file)}<br><b>Status: Load Error</b>"
                image_widget.value = b'' # Clear image widget
                # DO NOT automatically call display_next_suspect here on load error
                return # Exit function for this problematic file

            print("  Image loaded successfully.") # DEBUG

            # Add black border
            print("  Adding border...") # DEBUG
            img_bordered = cv2.copyMakeBorder(
                img, BORDER_SIZE, BORDER_SIZE, BORDER_SIZE, BORDER_SIZE,
                cv2.BORDER_CONSTANT, value=BORDER_COLOR
            )
            print("  Border added.") # DEBUG

            # Convert to display format (PNG bytes)
            print("  Encoding image...") # DEBUG
            success, img_encoded = cv2.imencode('.png', img_bordered)
            if not success:
                 print(f"❌ Error: cv2.imencode failed for {current_file}. Skipping.")
                 # Display buttons again to allow skipping
                 display(ui_layout)
                 delete_button.disabled = True
                 keep_button.disabled = False
                 filename_widget.value = f"<b>File:</b> {os.path.basename(current_file)}<br><b>Status: Encode Error</b>"
                 image_widget.value = b''
                 return

            img_bytes = img_encoded.tobytes()
            print("  Image encoded.") # DEBUG

            # Update the image widget
            image_widget.value = img_bytes
            filename_widget.value = f"<b>File:</b> {os.path.basename(current_file)}"

            # Display the widgets
            print("  Displaying UI layout...") # DEBUG
            display(ui_layout)
            delete_button.disabled = False
            keep_button.disabled = False
            print("  UI displayed.") # DEBUG

        except Exception as e:
            # Catch any other unexpected errors during display process
            print(f"❌❌ An unexpected error occurred displaying {current_file}:")
            traceback.print_exc() # Print detailed traceback
            print("  Skipping this file. Click 'Keep / Skip' to proceed.")
            # Display buttons again to allow skipping
            display(ui_layout)
            delete_button.disabled = True
            keep_button.disabled = False
            filename_widget.value = f"<b>File:</b> {os.path.basename(current_file)}<br><b>Status: Display Error</b>"
            image_widget.value = b''


# --- Button Click Handlers ---
def on_delete_button_clicked(b):
    """Handles the delete action."""
    global current_suspect_index, suspect_files
    print("\nDelete button clicked.") # DEBUG
    if 0 <= current_suspect_index < len(suspect_files):
        file_to_delete = suspect_files[current_suspect_index]
        try:
            if os.path.exists(file_to_delete):
                os.remove(file_to_delete)
                print(f"🗑️ Deleted: {file_to_delete}")
            else:
                print(f"Info: File already deleted: {file_to_delete}")
        except OSError as e:
            print(f"Error deleting file {file_to_delete}: {e}", file=sys.stderr)
            # Don't advance index automatically on error, let user retry or skip
            # Maybe disable delete button temporarily?
            delete_button.disabled = True
            return # Stop here

    # Trigger display of the next suspect directly (more reliable than eval_js here)
    display_next_suspect()


def on_keep_button_clicked(b):
    """Handles the keep/skip action."""
    print("\nKeep/Skip button clicked.") # DEBUG
    # Trigger display of the next suspect directly
    display_next_suspect()


# --- Create Widgets (define them early) ---
image_widget = widgets.Image(format='png', width=600)
filename_widget = widgets.HTML(value="")
delete_button = widgets.Button(description="Delete This File", button_style='danger', icon='trash')
keep_button = widgets.Button(description="Keep / Skip", button_style='info', icon='check')
buttons = widgets.HBox([delete_button, keep_button])
ui_layout = widgets.VBox([filename_widget, image_widget, buttons])
out = widgets.Output() # Output area widget

# --- Register Callbacks ---
delete_button.on_click(on_delete_button_clicked)
keep_button.on_click(on_keep_button_clicked)

# Note: We removed the JavaScript callback registration as we now call display_next_suspect directly

# --- Main Execution ---
if __name__ == "__main__":
    # Display the output area *first*
    display(out)

    # Find suspects only ONCE per kernel session unless explicitly reset
    if not files_found:
        with out: # Show finding process in the output area
            clear_output(wait=True)
            if not os.path.isdir(CHUNK_FOLDER):
                print(f"Error: Chunk folder not found at '{CHUNK_FOLDER}'", file=sys.stderr)
                # Stop execution if folder is missing
                suspect_files = [] # Ensure it's empty
            else:
                suspect_files = find_suspect_chunk_files(CHUNK_FOLDER, LOW_CONTRAST_THRESHOLD)
                current_suspect_index = -1 # Reset index

    # --- Initial Trigger ---
    # Directly call the function to display the first item *after* finding files
    # Ensure this happens within the 'out' context for the first display
    with out:
        if suspect_files: # Only attempt display if there are suspects
            print("\nStarting review process...")
            # Call the function directly to display the first image (index will become 0)
            display_next_suspect()
        elif files_found: # Files were scanned, but none were suspect
             print("No suspect files found to review.")
        # If folder wasn't found, error was already printed

Output()


Delete button clicked.
🗑️ Deleted: /content/drive/MyDrive/folder_partitioned_merged_clean/Commentaire de Leibniz (AGREG interne 2022 - note _ 16)/page_15_chunk_002.png

Entering display_next_suspect, new index: 1

Delete button clicked.
🗑️ Deleted: /content/drive/MyDrive/folder_partitioned_merged_clean/Commentaire de Leibniz (AGREG interne 2022 - note _ 16)/page_15_chunk_001.png

Entering display_next_suspect, new index: 2

Delete button clicked.
🗑️ Deleted: /content/drive/MyDrive/folder_partitioned_merged_clean/Commentaire de Mill (AGREG 2022 - note 12)/page_12_chunk_003.png

Entering display_next_suspect, new index: 3

Delete button clicked.
🗑️ Deleted: /content/drive/MyDrive/folder_partitioned_merged_clean/Commentaire de Mill (AGREG 2022 - note 12)/page_12_chunk_002.png

Entering display_next_suspect, new index: 4

Delete button clicked.
🗑️ Deleted: /content/drive/MyDrive/folder_partitioned_merged_clean/Commentaire Locke, Essay, II, 21 (AGREG ext. 2019 - note _ 16)/page_16_chunk_00

In [ ]:
import os

def list_files_in_directory(directory_path):
    """
    List all files in the specified directory.

    Args:
        directory_path (str): Path to the directory

    Returns:
        list: List of filenames in the directory
    """
    try:
        # Get the list of all files in the directory
        files = os.listdir(directory_path)

        # Print all the files
        print(f"Files in {directory_path}:")
        for file in files:
            # Get the full path
            full_path = os.path.join(directory_path, file)
            # Check if it's a file (not a directory)
            if os.path.isfile(full_path):
                print(f"- {file}")

        return files
    except FileNotFoundError:
        print(f"Directory not found: {directory_path}")
        return []
    except PermissionError:
        print(f"Permission denied to access: {directory_path}")
        return []

# Example usage
if __name__ == "__main__":
    # Replace with the path to your directory
    directory = "/content/folder_partitioned/Est-ce une chance de naître humain ? (AGREG interne 2023 - note : 13)"  # Windows example
    # directory = "/home/yourusername/documents"   # Linux/Mac example

    list_files_in_directory(directory)

Files in /content/folder_partitioned/Est-ce une chance de naître humain ? (AGREG interne 2023 - note : 13):
- page_06_chunk_001.png
- page_08_chunk_002.png
- page_01_chunk_001.png
- page_03_chunk_001.png
- page_13_chunk_002.png
- page_07_chunk_003.png
- page_07_chunk_002.png
- page_14_chunk_002.png
- page_12_chunk_000.png
- page_15_chunk_001.png
- page_02_chunk_002.png
- page_01_chunk_002.png
- page_07_chunk_001.png
- page_10_chunk_000.png
- page_13_chunk_001.png
- page_04_chunk_003.png
- page_08_chunk_003.png
- page_12_chunk_002.png
- page_05_chunk_001.png
- page_14_chunk_000.png
- page_08_chunk_000.png
- page_12_chunk_001.png
- page_11_chunk_001.png
- page_06_chunk_000.png
- page_05_chunk_002.png
- page_04_chunk_001.png
- page_11_chunk_003.png
- page_11_chunk_002.png
- page_15_chunk_000.png
- page_02_chunk_000.png
- page_10_chunk_001.png
- page_07_chunk_000.png
- page_02_chunk_001.png
- page_06_chunk_002.png
- page_05_chunk_000.png
- page_03_chunk_002.png
- page_15_chunk_003.png
- p

In [ ]:
import os
import re
from tqdm.notebook import tqdm # Progress bar for Colab/Jupyter

# --- Configuration ---
CHUNK_FOLDER = "/content/folder_partitioned" # Folder containing the subfolders with chunks
NUM_DIGITS = 5 # Number of digits for the new filenames (e.g., 5 -> 00001, 00002)

# --- Helper Function for Sorting ---
def sort_key_chunk_files(filename):
    """
    Generates a sort key for chunk filenames like 'basename_chunk_XXX.png'.
    Sorts primarily by the 'basename' part and secondarily by the chunk number.
    Handles potential variations or already renamed files less predictably.
    """
    base_name_part = filename # Default if pattern doesn't match
    chunk_number = 0 # Default chunk number

    # Use rsplit to split based on the *last* occurrence of '_chunk_'
    parts = filename.rsplit('_chunk_', 1)

    if len(parts) == 2:
        base_name_part = parts[0]
        chunk_num_str = parts[1].lower().replace('.png', '')
        try:
            chunk_number = int(chunk_num_str)
        except ValueError:
            # Couldn't convert chunk number part to int, assign a default or flag
            print(f"Warning: Could not parse chunk number from '{filename}'. Using default 0.")
            chunk_number = -1 # Assign -1 to sort these potentially problematic files first
    else:
        # Handle files that *don't* match the pattern (e.g., already renamed?)
        # If it's purely numeric (like '0001.png'), try to parse it.
        numeric_part = filename.lower().replace('.png', '')
        if numeric_part.isdigit():
            try:
                # Treat it as already potentially renamed, use its number for sorting
                chunk_number = int(numeric_part)
                # Assign an empty base name so these sort together, potentially first or last
                # depending on other base names. Let's aim for first.
                base_name_part = ""
            except ValueError:
                # Not purely numeric, keep defaults (sorts based on full filename)
                pass
        else:
             print(f"Warning: Filename '{filename}' doesn't match expected '_chunk_XXX.png' pattern. Sorting relies on filename string.")
             # Keep original filename as base_name_part, chunk_number as 0

    return (base_name_part, chunk_number)

# --- Main Renaming Logic ---
print(f"Starting sequential renaming in subfolders of: {CHUNK_FOLDER}")
print(f"New filenames will be padded to {NUM_DIGITS} digits (e.g., {1:0{NUM_DIGITS}d}.png)")

total_renamed = 0
total_skipped = 0
processed_folders = 0

# Get a list of subdirectories first for the progress bar
try:
    subfolders = sorted([os.path.join(CHUNK_FOLDER, d) for d in os.listdir(CHUNK_FOLDER) if os.path.isdir(os.path.join(CHUNK_FOLDER, d))])
    print(f"Found {len(subfolders)} subfolders to process.")
except FileNotFoundError:
    print(f"Error: Base directory '{CHUNK_FOLDER}' not found.")
    subfolders = []
except Exception as e:
    print(f"Error listing subfolders in '{CHUNK_FOLDER}': {e}")
    subfolders = []


if not subfolders:
    print("No subfolders found or error occurred. Exiting.")
else:
    for subfolder_path in tqdm(subfolders, desc="Processing Subfolders"):
        processed_folders += 1
        print(f"\nProcessing subfolder: {subfolder_path}")

        try:
            # Find only PNG files directly in this subfolder
            all_files = [f for f in os.listdir(subfolder_path)
                         if os.path.isfile(os.path.join(subfolder_path, f)) and f.lower().endswith(".png")]
        except Exception as e:
            print(f"  Error listing files in {subfolder_path}: {e}. Skipping this subfolder.")
            continue

        if not all_files:
            print("  No PNG files found in this subfolder.")
            continue

        # Sort files using the custom key to ensure correct page and chunk order
        try:
            sorted_files = sorted(all_files, key=sort_key_chunk_files)
            print(f"  Found {len(sorted_files)} PNG files. Sorted order (first few): {sorted_files[:5]}")
        except Exception as e:
             print(f"  Error sorting files in {subfolder_path}: {e}. Skipping renaming in this folder.")
             total_skipped += len(all_files)
             continue


        # --- Renaming Loop for this Subfolder ---
        subfolder_counter = 1 # Start counter at 1 for each new subfolder
        files_in_folder_renamed = 0
        files_in_folder_skipped = 0

        for current_filename in sorted_files:
            # Construct the new filename based on the counter
            new_filename = f"{subfolder_counter:0{NUM_DIGITS}d}.png"
            old_path = os.path.join(subfolder_path, current_filename)
            new_path = os.path.join(subfolder_path, new_filename)

            # Avoid renaming if the file already has the target name
            if current_filename == new_filename:
                # print(f"  Skipping '{current_filename}', already correctly named.")
                files_in_folder_skipped += 1
                # We still increment the counter, assuming this was the correct file for this position
                subfolder_counter += 1
                continue

            # Safety check: does the target new name already exist unexpectedly?
            # This indicates a potential flaw in sorting or duplicate counter use.
            if os.path.exists(new_path):
                print(f"  Error: Target file '{new_path}' already exists, but current file is '{current_filename}'. Skipping rename. Check sorting/duplicates.")
                files_in_folder_skipped += 1
                # Do not increment counter, as we skipped the intended rename for this position
                continue

            # Perform the rename
            try:
                os.rename(old_path, new_path)
                # print(f"  Renamed '{current_filename}' -> '{new_filename}'") # Optional verbose output
                files_in_folder_renamed += 1
                subfolder_counter += 1 # Increment counter *only after* successful rename
            except OSError as e:
                print(f"  Error renaming '{current_filename}' to '{new_filename}': {e}")
                files_in_folder_skipped += 1
                # Do not increment counter if rename failed

        print(f"  Subfolder Summary: Renamed {files_in_folder_renamed}, Skipped {files_in_folder_skipped}")
        total_renamed += files_in_folder_renamed
        total_skipped += files_in_folder_skipped

    print("\n--- Renaming Complete ---")
    print(f"Processed {processed_folders} subfolders.")
    print(f"Total files renamed: {total_renamed}")
    print(f"Total files skipped (already named, error, etc.): {total_skipped}")

Starting sequential renaming in subfolders of: /content/folder_partitioned
New filenames will be padded to 5 digits (e.g., 00001.png)
Found 26 subfolders to process.


Processing Subfolders:   0%|          | 0/26 [00:00<?, ?it/s]


Processing subfolder: /content/folder_partitioned/Est-ce une chance de naître humain ? (AGREG interne 2023 - note : 13)
  Found 52 PNG files. Sorted order (first few): ['page_01_chunk_000.png', 'page_01_chunk_001.png', 'page_01_chunk_002.png', 'page_02_chunk_000.png', 'page_02_chunk_001.png']
  Subfolder Summary: Renamed 52, Skipped 0

Processing subfolder: /content/folder_partitioned/Est-ce une chance de naître humain ? (AGREG interne 2023 - note : 16)
  Found 34 PNG files. Sorted order (first few): ['page_01_chunk_000.png', 'page_01_chunk_001.png', 'page_01_chunk_002.png', 'page_02_chunk_000.png', 'page_02_chunk_001.png']
  Subfolder Summary: Renamed 34, Skipped 0

Processing subfolder: /content/folder_partitioned/Est-ce une chance de naître humain ? (AGREG interne 2023 - note : 18,5)
  Found 78 PNG files. Sorted order (first few): ['page_01_chunk_000.png', 'page_01_chunk_001.png', 'page_01_chunk_002.png', 'page_02_chunk_000.png', 'page_02_chunk_001.png']
  Subfolder Summary: Ren